# Load standard packages and pipeline.py

In [5]:
import pandas as pd
import numpy as np
import os
import re
import sqlalchemy
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as sm

import matplotlib as mpl
%matplotlib inline

In [6]:
os.chdir('/home/matt/Git/PGIP/config/cache')
import imputers as imp

### Set display options

In [7]:
pd.set_option('max_colwidth', 200)
pd.set_option('display.width',200)

# Import the pipeline

In [8]:
os.chdir('..')
os.chdir('..')
os.chdir('src')
import pipeline

In [14]:
os.chdir('/home/matt/Git/PGIP')
#os.chdir('/home/matt/Datasets/Battelle Data/ingestion')

# Initialize with config files

In [16]:
P = pipeline.Pipeline('config/cache','log')
E = pipeline.Explorer('config/cache')

INFO:pipeline:Pipeline initialized successfully. configDir: /home/matt/Git/PGIP/config/cache dbConnection: Engine(sqlite:///database/main.db)


# Check out the config

In [17]:
print(P.extensions)
print(E.extensions)

['.csv', '.xls', '.xlsx']
['.csv', '.xls', '.xlsx']


In [18]:
P.dtypes
len(P.dtypes)

35

In [19]:
P.columnMaps

{'EV': {'Abbreviated source title': 'AbbrPubTitle',
  'Abstract': 'Abstract',
  'Author': 'Authors',
  'Author affiliation': 'Affiliations',
  'CODEN': 'CODEN',
  'Controlled/Subject terms': 'SourceCuratedKeywords',
  'DOI': 'DOI',
  'Document type': 'Type',
  'ISBN13': 'ISBN',
  'ISSN': 'ISSN',
  'Issue': 'Issue',
  'Issue date': 'PublicationDate',
  'Language': 'OriginalLanguage',
  'Number of references': 'ReferenceCount',
  'Pages': 'PageRange',
  'Publication year': 'Year',
  'Publisher': 'Publisher',
  'Source': 'PublicationTitle',
  'Title': 'Title',
  'Uncontrolled terms': 'GeneratedKeywords',
  'Volume': 'Volume'},
 'IEEE': {'Abstract': 'Abstract',
  'Article Citation Count': 'CitationCount',
  'Author Affiliations': 'Affiliations',
  'Author Keywords': 'AuthorKeywords',
  'Authors': 'Authors',
  'DOI': 'DOI',
  'Document Title': 'Title',
  'End Page': 'PageEnd',
  'INSPEC Controlled Terms': 'SourceCuratedKeywords',
  'INSPEC Non-Controlled Terms': 'GeneratedKeywords',
  'ISBN

In [9]:
P.newColumns

['AuthorsWithAffiliations', 'PageCount', 'CitationCount', 'Month', 'TimeSlice']

In [10]:
print(dir(P.cleaners))
print(dir(P.fixers))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'castTypes', 'cleanAbbrPubTitle', 'cleanCODEN', 'cleanISBN', 'cleanISSN', 'cleanIssue', 'cleanType', 'cleanVolume', 'cleanYear', 'dt', 'isnull', 'np', 're', 'to_datetime']
['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'fixScopus', 'np', 're']


In [11]:
print(P.dbConnection)
print(P.dbConfig)

Engine(sqlite:////home/matt/Datasets/Battelle Data/database/main.db)
{'tables': ['main'], 'engine': 'sqlite', 'hostname': '', 'location': '/home/matt/Datasets/Battelle Data/database/main.db'}


# try reading a sample dir with Explorer:

In [20]:
E.readSampleData('/home/matt/Git/PGIP/sandbox')

Data files: ['ISI_Medline-synthetic biology.csv', 'ISI_Medline-transcriptomics.csv']
column sample: ['PT' 'AN' 'DT' 'TI' 'FT' 'AU' 'CA' 'SO' 'VL' 'IS']
column sample: ['PT' 'AN' 'DT' 'TI' 'FT' 'AU' 'CA' 'SO' 'VL' 'IS']
Data files: ['ISI_Inspec-proteomics.csv', 'ISI_Inspec-transcriptomics.csv']
column sample: ['PT' 'AN' 'DT' 'PN' 'PC' 'TI' 'AU' 'ED' 'TR' 'OP']
column sample: ['PT' 'AN' 'DT' 'PN' 'PC' 'TI' 'AU' 'ED' 'TR' 'OP']
Data files: ['IEEE-proteomics.csv', 'IEEE-transcriptomics.csv']
column sample: ['Document Title' 'Authors' 'Author Affiliations' 'Publication Title'
 'Date Added To Xplore' 'Year' 'Volume' 'Issue' 'Start Page' 'End Page']
column sample: ['Document Title' 'Authors' 'Author Affiliations' 'Publication Title'
 'Date Added To Xplore' 'Year' 'Volume' 'Issue' 'Start Page' 'End Page']
Data files: ['EV-transcriptomics.xlsx', 'EV-proteomics.2013-2016.xlsx']
column sample: ['Title' 'Accession number' 'Author' 'Author affiliation'
 'Corresponding author' 'Source' 'Abbreviated 

### run explorer cleaning process

In [21]:
E.renameColumns()

# try reading a sample file with Pipeline:

In [14]:
os.getcwd()

'/home/matt/Datasets/Battelle Data/ingestion'

In [15]:
d = P.readRawFile('/home/matt/Datasets/Battelle Data/raw_data/EngineeringVillage/proteomics.2013-2016.xlsx')

INFO:pipeline:/home/matt/Datasets/Battelle Data/raw_data/EngineeringVillage/proteomics.2013-2016.xlsx read correctly. Source: EV Confidence: 0.957 Rows: 434


In [22]:
d = P.readRawFile('/home/matt/Git/PGIP/sandbox/ISI_CCC/ISI_CCC-proteomics.csv')

INFO:pipeline:/home/matt/Git/PGIP/sandbox/ISI_CCC/ISI_CCC-proteomics.csv read correctly. Source: ISI_CCC Confidence: 1.0 Rows: 19


### try to upload pipleline with Pipeline (fix,clean,rename,upload):

In [26]:
print(d.fixed)
print(d.cleaned)
print(d.renamed)

False
False
False


In [27]:
d = P.fix(d)
print(d.fixed)
d = P.renameColumns(d)
print(d.renamed)
d = P.clean(d)
print(d.cleaned)

True
True
True


In [31]:
d = P.addColumns(d)

In [32]:
d.data.columns

Index(['Authors', 'Title', 'PublicationTitle', 'OriginalLanguage', 'Type', 'AuthorKeywords', 'SourceCuratedKeywords', 'Abstract', 'AuthorsWithAffiliations', 'Publisher', 'ISSN', 'ISBN',
       'AbbrPubTitle', 'Year', 'PublicationDate', 'Volume', 'Issue', 'PageStart', 'PageEnd', 'PageCount', 'DOI', 'CODEN', 'EID', 'TimeSlice', 'MESHTerms', 'Affiliations', 'References',
       'GeneratedKeywords', 'ReferenceCount', 'FundingSources', 'PubMedID', 'Month', 'PageRange', 'CitationCount', 'Source'],
      dtype='object')

In [30]:
d.data

,Authors,Title,PublicationTitle,OriginalLanguage,Type,AuthorKeywords,SourceCuratedKeywords,Abstract,AuthorsWithAffiliations,Publisher,...,Affiliations,References,GeneratedKeywords,ReferenceCount,FundingSources,PubMedID,Month,PageRange,CitationCount,Source
0,"Zhang, Xinyu; Wang, Keyi",Double-incident angle technique for surface plasmon resonance measurements,OPTICS COMMUNICATIONS,English,article,"Surface plasmon resonance, Double-incident angle, Differential measurement, Refractive index",NaN,A new double-incident angle technique for surface plasmon resonance measurement is described. It is based on differential measurements at two chosen incident angles where the slopes are steepest a...,None,ELSEVIER SCIENCE BV,...,NaN,NaN,NaN,NaN,NaN,NaN,9,NaN,None,ISI_CCC
1,"Chen, Qi; Yan, Guoquan; Zhang, Xiangmin",Applying multiple proteases to direct digestion of hundred-scale cell samples for proteome analysis,RAPID COMMUNICATIONS IN MASS SPECTROMETRY,English,article,NaN,"MASS-SPECTROMETRY, PROTEINS, CAPTURE",RationaleAnalyzing the proteome on the scale of only several hundred cells with mass spectrometry has great significance for applications with limited sample amounts. We applied multiple proteases...,None,WILEY-BLACKWELL,...,NaN,NaN,NaN,NaN,NaN,NaN,8,NaN,None,ISI_CCC
2,"Xia, Simin; Yuan, Huiming; Chen, Yuanbo; Liang, Zheng; Zhang, Lihua; Zhang, Yukui",Integrated SDS removal and protein digestion by hollow fiber membrane based device for SDS-assisted proteome analysis,TALANTA,English,article,"SDS removal, On-line digestion, Hollow fiber membrane interface, Immobilized enzymatic reactor, Proteome analysis","SAMPLE PREPARATION METHOD, TANDEM MASS-SPECTROMETRY, LIQUID-CHROMATOGRAPHY, GEL-ELECTROPHORESIS, PEPTIDE SEPARATION, IDENTIFICATION","In this work, a novel integrated sample preparation device for SDS-assisted proteome analysis was developed, by which proteins dissolved in 4% (w/v) SDS were first diluted by 50% methanol, and the...",None,ELSEVIER SCIENCE BV,...,NaN,NaN,NaN,NaN,NaN,NaN,8,NaN,None,ISI_CCC
3,"Yan, Kun; Fu, Zongming; Yang, Chen; Zhang, Kai; Jiang, Shanshan; Lee, Dong-Hoon; Heo, Hye-Young; Zhang, Yi; Cole, Robert N.; Van Eyk, Jennifer E.; Zhou, Jinyuan",Assessing Amide Proton Transfer (APT) MRI Contrast Origins in 9 L Gliosarcoma in the Rat Brain Using Proteomic Analysis,MOLECULAR IMAGING AND BIOLOGY,English,article,"APT imaging, CEST imaging, Glioma, Mobile protein, Proteomics","MAGNETIZATION-TRANSFER, GLIOBLASTOMA-MULTIFORME, MOBILE PROTEINS, TUMORS, RESONANCE, GLIOMA, SATURATION, GRADE, VIVO, PEPTIDES",To investigate the biochemical origin of the amide photon transfer (APT)-weighted hyperintensity in brain tumors.Seven 9 L gliosarcoma-bearing rats were imaged at 4.7 T. Tumor and normal brain tis...,None,SPRINGER,...,NaN,NaN,NaN,NaN,NaN,NaN,8,NaN,None,ISI_CCC
4,"Fan, Longquan; Wu, Xiaolei; Tian, Zhen; Jia, Kaizhi; Pan, Yinghong; Li, Jingrui; Gao, Hongbo",Comparative proteomic analysis of gamma-aminobutyric acid responses in hypoxia-treated and untreated melon roots,PHYTOCHEMISTRY,English,article,"Melon roots (Cucumis melon L. Cucurbitaceae), GABA, Hypoxia stress, Proteomics","ELONGATION-FACTOR 1A, TRANSLATION ELONGATION, ACTIN CYTOSKELETON, EXOGENOUS CALCIUM, STRESS-RESPONSE, CUCUMBER ROOTS, PROTEINS, PLANTS, TOLERANCE, GABA",Hypoxia is one of the main environmental stresses that accounts for decreasing crop yield. To further investigate the mechanisms whereby exogenous GABA alleviates hypoxia injury to melon seedlings...,None,PERGAMON-ELSEVIER SCIENCE LTD,...,NaN,NaN,NaN,NaN,NaN,NaN,8,NaN,None,ISI_CCC
5,"Shi, Haitao; Wang, Xin; Tan, Dun-Xian; Reiter, Russel J.; Chan, Zhulong",Comparative physiological and proteomic analyses reveal the actions of melatonin in the reduction of oxidative stress in Bermuda grass (Cynodon dactylon (L). Pers.),JOURNAL OF PINEAL RESEARCH,English,article,"Bermuda grass, iTRAQ (isobaric tags for relative and absolute quantitation), melatonin, oxidative stress, proteomic, reactive oxygen speci

In [21]:


#for column in d.data.columns:
#    print(column)
#    print(d.data[column].apply(type).unique())

### try uploading the first i columns for every i:

In [33]:
for i in range(1,len(d.data.columns)+1):
    try:
        d.data.loc[:,d.data.columns[0:i]].to_sql('main',P.dbConnection,if_exists='replace',index=False,dtype=P.dtypes)
    except:
        print("problem with {1}".format(d.data.columns[i-1]))

# Check that the upload was successful

In [21]:
fromdb = pd.read_sql('SELECT * FROM main',P.dbConnection)

### do the uploaded/downloaded copy and the original have the same columns?

In [22]:
print(set(d.data.columns).difference(fromdb.columns))
print(set(fromdb.columns).difference(d.data.columns))

set()
set()


### do they have the same null counts?

In [23]:
fromdb.apply(lambda x: x.isnull().sum())

Authors                     0
Title                       0
PublicationTitle            0
OriginalLanguage            0
Type                        0
AuthorKeywords              6
SourceCuratedKeywords       1
Abstract                    0
AuthorsWithAffiliations    19
Publisher                   0
ISSN                        0
ISBN                       19
AbbrPubTitle                0
Year                        0
PublicationDate             1
Volume                      0
Issue                       9
PageStart                   0
PageEnd                     0
PageCount                   0
DOI                        19
EID                        19
Affiliations               19
Month                       1
PageRange                  19
CODEN                      19
CitationCount              19
MESHTerms                  19
PubMedID                   19
FundingSources             19
TimeSlice                   1
Source                      0
GeneratedKeywords          19
ReferenceC

In [24]:
d.data.apply(lambda x: x.isnull().sum())

Authors                     0
Title                       0
PublicationTitle            0
OriginalLanguage            0
Type                        0
AuthorKeywords              6
SourceCuratedKeywords       1
Abstract                    0
AuthorsWithAffiliations    19
Publisher                   0
ISSN                        0
ISBN                       19
AbbrPubTitle                0
Year                        0
PublicationDate             1
Volume                      0
Issue                       9
PageStart                   0
PageEnd                     0
PageCount                   0
DOI                        19
EID                        19
Affiliations               19
Month                       1
PageRange                  19
CODEN                      19
CitationCount              19
MESHTerms                  19
PubMedID                   19
FundingSources             19
TimeSlice                   1
Source                      0
GeneratedKeywords          19
ReferenceC

# Try uploading a whole directory with Pipeline

In [34]:
os.chdir('/home/matt/Datasets/Battelle Data/')
os.getcwd()

'/home/matt/Datasets/Battelle Data'

In [35]:
P.ingestDir('raw_data','main')

INFO:pipeline:Beginning ingestion: /home/matt/Datasets/Battelle Data/raw_data recursive: True database: /home/matt/Datasets/Battelle Data/database/main.db table: main
INFO:pipeline:raw_data/Scopus/minimal genome.xlsx read correctly. Source: Scopus Confidence: 1.0 Rows: 10
INFO:pipeline:raw_data/Scopus/minimal genome.xlsx uploaded successfully. table: main
INFO:pipeline:raw_data/Scopus/biomimetics.2008-2005_Scopus.xlsx read correctly. Source: Scopus Confidence: 1.0 Rows: 1440
INFO:pipeline:raw_data/Scopus/biomimetics.2008-2005_Scopus.xlsx uploaded successfully. table: main
INFO:pipeline:raw_data/Scopus/genomics.2012-2009_Scopus.xlsx read correctly. Source: Scopus Confidence: 1.0 Rows: 1798
INFO:pipeline:raw_data/Scopus/genomics.2012-2009_Scopus.xlsx uploaded successfully. table: main
INFO:pipeline:raw_data/Scopus/artificial nucleic acid.xlsx read correctly. Source: Scopus Confidence: 1.0 Rows: 256
INFO:pipeline:raw_data/Scopus/artificial nucleic acid.xlsx uploaded successfully. table: m

# Check out the adders and fixers

In [27]:
print(E.data['Scopus'].data.shape)

(533, 284)


In [28]:
E.renameColumns()

In [30]:
for newColumn in P.newColumns:
    for source in E.sources:
        a = E.sampleColumns(source,P.newColumns[newColumn],n=20)
        a['Source']=source
        print(a)
        adder = eval('P.adders.add'+newColumn)
        a[newColumn] = a.apply(adder,axis=1)
        print(a)
        x = input('')

   PageStart PageEnd  PageRange PageCount  Source
0        NaN     NaN        NaN       NaN  Scopus
1       8477    8480        NaN       NaN  Scopus
2        NaN     NaN        NaN       NaN  Scopus
3        111     125        NaN       NaN  Scopus
4        269     274        NaN       NaN  Scopus
5        265     270        NaN       NaN  Scopus
6          3      21        NaN       NaN  Scopus
7        703     709        NaN       NaN  Scopus
8       1010    1013        NaN       NaN  Scopus
9        347     354        NaN       NaN  Scopus
10       356     361        NaN       NaN  Scopus
11       NaN     NaN        NaN        12  Scopus
12       167     172        NaN       NaN  Scopus
13         1      12        NaN       NaN  Scopus
14       898     905        NaN       NaN  Scopus
15       851     855        NaN       NaN  Scopus
16       204     208        NaN       NaN  Scopus
17      2641    2648        NaN       NaN  Scopus
18        95     102        NaN       NaN  Scopus


In [29]:
E.sampleSources('PageCount',n=20)

,EV,IEEE,ISI_CCC,ISI_Inspec,ISI_Medline,ISI_WoS,Scopus
0,NaN,NaN,20,NaN,NaN,9,NaN
1,NaN,NaN,13,NaN,NaN,41,NaN
2,NaN,NaN,11,NaN,NaN,13,NaN
3,NaN,NaN,12,NaN,NaN,10,NaN
4,NaN,NaN,9,NaN,NaN,11,NaN
5,NaN,NaN,6,NaN,NaN,1,NaN
6,NaN,NaN,8,NaN,NaN,9,NaN
7,NaN,NaN,11,NaN,NaN,5,NaN
8,NaN,NaN,15,NaN,NaN,12,NaN
9,NaN,NaN,4,NaN,NaN,10,NaN


In [30]:
E.sampleSources('PageRange',n=20)

,EV,IEEE,ISI_CCC,ISI_Inspec,ISI_Medline,ISI_WoS,Scopus
0,14466-14472,NaN,NaN,S9 (9 pp.),12737-52,NaN,NaN
1,NaN,NaN,NaN,2991-5,28-42,NaN,NaN
2,77-84,NaN,NaN,125-30,e0124560,NaN,NaN
3,651-664,NaN,NaN,287-8,NaN,NaN,NaN
4,116-123,NaN,NaN,221-6,917292,NaN,NaN
5,472-480,NaN,NaN,503-19,121-6,NaN,NaN
6,307-314,NaN,NaN,674-7,1973-8,NaN,NaN
7,7446-7454,NaN,NaN,1280-3,39-48,NaN,NaN
8,1172-1186,NaN,NaN,121 (11 pp.),141-52,NaN,NaN
9,931-943,NaN,NaN,386-91,47-52,NaN,NaN


In [31]:
E.sampleSources('PageStart',n=20)

,EV,IEEE,ISI_CCC,ISI_Inspec,ISI_Medline,ISI_WoS,Scopus
0,NaN,1,235,3042,NaN,NaN,1711
1,NaN,192,449,904,NaN,W358,285
2,NaN,2195,99,045005 (13 pp.),NaN,NaN,358
3,NaN,117,140,065701 (10 pp.),NaN,169,925
4,NaN,1,3743,1045,NaN,710,1778
5,NaN,514,74,221,NaN,99,2325
6,NaN,1,1547,5659,NaN,1274,2663
7,NaN,1,11,49,NaN,422,2377
8,NaN,84,260,149,NaN,336,4
9,NaN,127,4931,5244,NaN,12944,986


In [32]:
E.sampleSources('PageEnd',n=20)

,EV,IEEE,ISI_CCC,ISI_Inspec,ISI_Medline,ISI_WoS,Scopus
0,NaN,663,238,S3 (18 pp.),NaN,1006,997
1,NaN,75,37,427,NaN,1497,533
2,NaN,41,80,2 pp.,NaN,230,237
3,NaN,962,428,453 (8 pp.),NaN,1525,214
4,NaN,703,131,143,NaN,6193,NaN
5,NaN,4,67,4580,NaN,1283,335
6,NaN,225,1394,677,NaN,301,58
7,NaN,4,99,57,NaN,NaN,580+607
8,NaN,5,487,2544,NaN,948,460
9,NaN,5,2219,496,NaN,343,337


In [29]:
E.sampleSources('CitationCount',n=20)

,EV,IEEE,ISI_CCC,ISI_Inspec,ISI_Medline,ISI_WoS,Scopus
0,NaN,0,NaN,NaN,NaN,19,NaN
1,NaN,0,NaN,NaN,NaN,12,1
2,NaN,0,NaN,NaN,NaN,204,11
3,NaN,0,NaN,NaN,NaN,19,NaN
4,NaN,2,NaN,NaN,NaN,52,5
5,NaN,0,NaN,NaN,NaN,25,2
6,NaN,0,NaN,NaN,NaN,63,8
7,NaN,4,NaN,NaN,NaN,21,NaN
8,NaN,0,NaN,NaN,NaN,11,6
9,NaN,2,NaN,NaN,NaN,72,15


In [33]:
a = E.sampleSources('ISBN',n=20)
print(a)
print(a.apply(lambda x: x.apply(P.cleaners.cleanISBN)))

    EV               IEEE  ISI_CCC         ISI_Inspec  ISI_Medline            ISI_WoS         Scopus
0  NaN                NaN      NaN                NaN          NaN                NaN            NaN
1  NaN                NaN      NaN  978-1-4577-0887-9          NaN                NaN            NaN
2  NaN  978-0-7695-3745-0      NaN                NaN          NaN                NaN            NaN
3  NaN                NaN      NaN  978-1-4799-5669-2          NaN                NaN            NaN
4  NaN  978-1-4244-8303-7      NaN  978-1-4673-2164-8          NaN  978-0-12-374621-4            NaN
5  NaN  978-1-61284-723-8      NaN                NaN          NaN                NaN            NaN
6  NaN  978-0-7695-3656-9      NaN                NaN          NaN                NaN            NaN
7  NaN                NaN      NaN                NaN          NaN                NaN            NaN
8  NaN  978-1-4244-2901-1      NaN  978-1-4799-0248-4          NaN                NaN      

In [34]:
a = E.sampleSources('ISSN',n=20)
print(a)
print(a.apply(lambda x: x.apply(P.cleaners.cleanISBN)))

          EV       IEEE    ISI_CCC ISI_Inspec ISI_Medline    ISI_WoS  \
0   01677799        NaN  0742-3098        NaN   1932-6203  0141-5492   
1   09593993        NaN  0250-7005  2050-750X   0253-9772  0344-5704   
2   14337851        NaN  1553-7390  0927-7765   1422-0067  0815-9319   
3   02777037  2151-7614  1615-9853  0142-9612   1471-2350  0305-1048   
4   09445013        NaN  0958-1669  1874-3919   1758-0463        NaN   
5   10070214  2151-7614  1664-462X  1545-5963   2095-0225  1615-9853   
6   00405752  2151-7614  1664-462X  1536-1241   1676-5680  1932-6203   
7   20507518  2151-7614  2041-1723  1471-2105   1471-2229  0027-8424   
8   00405752  2151-7614  0032-0889  0028-0836   2095-0225  0167-4412   
9        NaN        NaN  0014-4894        NaN   1009-2137  1672-9072   
10  01676857  1545-5963  0031-9422  1536-1241   0513-4870  1001-0602   
11       NaN        NaN  1553-7366  1748-3387   0253-9772  0168-1656   
12  00218561  2151-7614  0048-3575  0956-7135   2210-3244  1532-

In [31]:
a = E.sampleSources('Type',n=20)
print(a)
print(a.apply(lambda x: x.apply(P.cleaners.cleanType)))

                      EV  IEEE             ISI_CCC        ISI_Inspec                                                                  ISI_Medline     ISI_WoS            Scopus
0   Journal article (JA)   NaN             Article  Conference Paper                                                              Journal Article      Letter           Article
1   Journal article (JA)   NaN             Article  Conference Paper                                                              Journal Article     Article           Article
2   Journal article (JA)   NaN             Article     Journal Paper                            Journal Article; Research Support, Non-U.S. Gov't     Article           Article
3   Journal article (JA)   NaN             Article  Conference Paper                                            English Abstract; Journal Article     Article  Conference Paper
4   Journal article (JA)   NaN              Review  Conference Paper                            Journal Article; Researc

In [44]:
a = E.sampleSources('Year',n=20)
print(a)
print(a.apply(lambda x: x.apply(P.cleaners.cleanYear)))

      EV  IEEE  ISI_CCC  ISI_Inspec  ISI_Medline  ISI_WoS Scopus
0   2013  2011     2015        2014         2015     2008   2013
1   2012  2011     2015        2014         2013     2007   2007
2   2014  2010     2015        2013         2015     2005   2015
3   2015  2010     2015        2009         2015     2008   2008
4   2014  2010     2015        2013         2014     2007   2007
5   2015  2011     2015        2013         2015     2005   2005
6   2013  2012     2015        2012         2015     2005   2010
7   2013  2013     2015        2013         2012     2008   2011
8   2013  2010     2015        2013         2013     2008   2014
9   2013  2012     2015        2014         2012     2008   2013
10  2012  2010     2015        2012         2015     2008   2007
11  2012  2014     2015        2014         2011     2008   2011
12  2012  2009     2015        2014         2011     2006   2007
13  2014  2011     2015        2014         2010     2008   2011
14  2015  2011     2015  

In [22]:
a = E.sampleSources('PublicationDate',n=20)
print(a)
for source in E.sources:
    b = a[[source]]
    b.columns = ['PublicationDate']
    b.loc[:,'Source'] = source
    b.loc[:,'Month'] = (b.apply(P.adders.addMonth, axis=1))
    print(b)

                   EV              IEEE ISI_CCC ISI_Inspec                  ISI_Medline ISI_WoS  Scopus
0                2010   24-26 Dec. 2011     JUN      7-Mar                     2009 Jan   7-Apr     NaN
1     October 2, 2012    26-31 May 2013     JUN       2013     2013  (Epub 2013 May 20)     MAR     NaN
2                2014   18-20 June 2010     JUL     26-Mar                         2015     OCT     NaN
3           June 2014     2-5 Nov. 2014     AUG      1-Apr                         2014     NOV     NaN
4       February 2013   11-13 June 2009     JUL     15-May                         2014     AUG     NaN
5       July 28, 2013   18-20 June 2010     JUN       Aug.                     2013 Jan     AUG     NaN
6      September 2013    10-12 May 2011     JUN    1 Sept.                         2015  22-Jun     NaN
7                2014  Jan.-Feb. 1 2015     AUG     1 Jan.                     2013 Feb     FEB     NaN
8            May 2014   22-24 June 2009     AUG       Dec.      

/home/matt/anaconda3/lib/python3.4/site-packages/pandas/core/indexing.py:266: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/matt/anaconda3/lib/python3.4/site-packages/pandas/core/indexing.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [20]:
a = E.sampleSources('Volume',n=20)
print(a)
print(a.apply(lambda x: x.apply(P.cleaners.cleanVolume)))

      EV  IEEE  ISI_CCC ISI_Inspec  ISI_Medline ISI_WoS Scopus
0     10     6        6         12           19      45     13
1    NaN   NaN       13         39         2013       7      8
2     34    25       62        NaN           10      15     24
3    125   NaN      170         32           14      43    NaN
4     80   NaN       51        102           28      49    NaN
5     45   NaN       15         13           14      71     24
6     78    10      168         33           31      54    419
7    NaN   NaN      365         15           29      11     22
8    148   NaN       29         14           37      35     19
9    407   NaN       17         75            9      19    815
10    36   NaN        6        205         2009      50      4
11   290   NaN      122        340            8       9     14
12    21   NaN       15        109           16       2     55
13   351   NaN        6         45            5      45     76
14     8   NaN       14        NaN            9       9

In [49]:
E.data['Scopus'].data.sample(10)

,AbbrPubTitle,Abstract,Affiliations,AuthorKeywords,Authors,AuthorsWithAffiliations,CODEN,CitationCount,DOI,Type,...,References,PublicationTitle,Title,Volume,Year,SourceCuratedKeywords,PublicationDate,MESHTerms,ReferenceCount,PageRange
316,Biotechnol. Lett.,"A recombinant baculovirus was constructed to simultaneously express codon-optimized virus-like particles (VLP), A VP1-2A-VP3 and VP0 of serotype O foot-and-mouth disease virus (FMDV), from individ...","Lanzhou Veterinary Research Institute of Chinese Academy of Agriculture Science, State Key Lab. of Veterinary Etiological Biology, National Foot-and-Mouth Disease Reference Lab., Key Laboratory of...",Baculovirus; Codon optimization; Foot-and-mouth disease virus; Virus-like particles,"Cao Y., Sun P., Fu Y., Bai X., Tian F., Liu X., Lu Z., Liu Z.","Cao, Y., Lanzhou Veterinary Research Institute of Chinese Academy of Agriculture Science, State Key Lab. of Veterinary Etiological Biology, National Foot-and-Mouth Disease Reference Lab., Key Labo...",BILED,7,10.1007/s10529-010-0295-8,Article,...,"Abrams, C.C., King, A.M.Q., Belsham, G.J., Assembly of foot-and-mouth disease virus empty capsids synthesized by vaccinia virus expression system (1995) J Gen Virol, 76, pp. 3089-3098; Belsham, G....",Biotechnology Letters,Formation of virus-like particles from O-type foot-and-mouth disease virus in insect cells using codon-optimized synthetic genes,32,2010,NaN,NaN,NaN,NaN,NaN
159,Pharm. Biotechnol.,"To compare the RNAi effects of siRNA which was prepared by chemical synthesis, siRNA expression vectors and siRNA expression cassettes. Constructing Lamin-siRNA-pFIV plasmid vector and lamin-siRNA...","China Pharmaceutical University, School of Life Science and Technology, Nanjing 210009, China; Biomics Biotechnologies (Nantong) Co. Ltd., Jiangsu Nantong 226001, China",Preparation methods; RNA interference; Silence efficiency; SiRNA,"Chen Y.-Q., Lu Y.-X., Wu M.-H., Gong T.-J., Zhu Y.-Y., Li Q.","Chen, Y.-Q., China Pharmaceutical University, School of Life Science and Technology, Nanjing 210009, China; Lu, Y.-X., Biomics Biotechnologies (Nantong) Co. Ltd., Jiangsu Nantong 226001, China; Wu...",NaN,NaN,NaN,Article,...,"Chinese sourceChinese sourceJulie, C., Sheng, D., Generation of RNAi Libraries for High-Throughput Screens (2006) Journal of Biomedicine and Biotechnology, p. 1. , J 45716; Wang, S., Shi, Z., Liu,...",Pharmaceutical Biotechnology,Comparison of the RNAi effects of SiRNA prepared by three methods,16,2009,NaN,NaN,NaN,NaN,NaN
499,"BIC-TA - Proc., Int. Conf. Bio-Inspired Comput.: Theor. Appl.","DNA computing is a bio-computing paradigm based on the specific hybridization of DNA strands. Most of DNA computing models and experiments were held in vitro. In this paper, a DNA computing paradi...","Institute of Software, School of Electronics Engineering and Computer Science, Peking University, Beijing, China; Key Laboratory of High Confidence Software Technologies (Peking University), Minis...",NaN,"Liu X., Wang S., Qiang X.","Liu, X., Institute of Software, School of Electronics Engineering and Computer Science, Peking University, Beijing, China, Key Laboratory of High Confidence Software Technologies (Peking Universit...",NaN,NaN,10.1109/BICTA.2009.5338084,Conference Paper,...,"Adleman, L., Molecular Computation of Solution to Combinatorial problems (1994) Science, 66 (11), pp. 1021-1024; Richard, J., Lipton, D.N.A., Solution of Hard Computational Problems (1995) Science...","BIC-TA 2009 - Proceedings, 2009 4th International Conference on Bio-Inspired Computing: Theories and Applications",Development of an in vivo computer for 3-SAT Problem,NaN,2009,NaN,NaN,NaN,NaN,NaN
25,ChemBioChem,Natural product peptide-based proteasome inhibitors show great potential as anticancer drugs. Here we have cloned the biosynthetic gene cluster of a potent proteasome inhibitor - glidobactin from ...,"Department of Microbial Natural Products, Helmholtz Institute for Pharmaceutical Research Saarland (HIPS), He

# Check out the concatenated DB

In [8]:
df = pd.read_sql('SELECT * FROM main',P.dbConnection)

In [9]:
df.shape

(2382, 34)

In [10]:
len(P.dtypes)

34

In [21]:
df.Month.sample(20)

1168   NaN
1045   NaN
1030   NaN
149     11
2247   NaN
406      7
476     10
360    NaN
1644     5
601      9
2317   NaN
1879    11
2332     8
1997   NaN
1259   NaN
1135   NaN
1005   NaN
783    NaN
1256   NaN
278      5
Name: Month, dtype: float64

In [50]:
df.loc[:,['CODEN','AbbrPubTitle','Volume','Issue','Month']].sample(20)

,CODEN,AbbrPubTitle,Volume,Issue,Month
160,ESTHAG,env sci techno,47,16,8
1376,None,NaN,NaN,NaN,6
1448,None,NaN,NaN,NaN,9
839,BTERD,biol tra ele res,144,1,NaN
333,JCRAEY,j chr a,1316,NaN,11
982,JIXUD,jis xue,28,10,NaN
1731,PPCPFQ,NaN,14,16,NaN
601,None,ana met,6,18,9
2211,None,NaN,45,4,8
977,None,mol biotec,32,3,NaN


## densities of non-null values

In [28]:
df.apply(lambda x: round(x.isnull().sum()/float(df.shape[0]),3))

Authors                    0.000
Title                      0.000
PublicationTitle           0.002
OriginalLanguage           0.094
Type                       0.095
AuthorKeywords             0.610
SourceCuratedKeywords      0.521
Abstract                   0.005
AuthorsWithAffiliations    1.000
Publisher                  0.366
ISSN                       0.000
ISBN                       0.851
AbbrPubTitle               0.388
Year                       0.000
PublicationDate            0.233
Volume                     0.146
Issue                      0.338
PageStart                  0.520
PageEnd                    0.520
PageCount                  0.135
DOI                        0.152
PubMedID                   0.626
PageRange                  0.457
References                 0.700
FundingSources             0.965
MESHTerms                  0.887
EID                        0.779
CODEN                      0.617
Month                      0.416
Source                     0.000
Affiliatio

In [46]:
def cleanAbbrPubTitle(string):
    if pd.isnull(string):
        return np.nan

    x = re.sub('[-\.,_\/]','',str(string)).lower()
    x = re.sub('(\s|^)(bio|chem|tox|gen|nano|tech|micro|med)', '\\1\\2X ',str(x))
    x = re.sub('(X (bio|chem|tox|gen|nano|tech|micro|med))', '\\1X ',str(x))
    x = re.sub('([^ X]{3,5}X|[\w]{3,3})\w*','\\1',x)
    x = re.sub('(bio|chem|tox|gen|nano|tech|micro|med)X ','\\1',x)
    x = re.sub('\s+',' ',x)
    x = re.sub('([\w]{6,6})\w*','\\1',x)
    return x



In [47]:
l = df.AbbrPubTitle
m = l.apply(cleanAbbrPubTitle)

In [48]:
for x in m.unique():
    #print(x)
    u = (l[m==x].unique())
    if len(u) > 1:
        print(u)

['Opt. Commun.' 'Opt Commun']
['BMC Bioinform.' 'BMC Bioinformatics']
['Microbiol Res.' 'Microbiol. Res.']
['IEEE Trans Signal Process' 'IEEE Trans. Signal Process.']
['Plant Cell Rep.' 'Plant Cell Reports']
['J. Biomed. Informatics' 'J. Biomed. Inform.']
['PLoS ONE' 'PLoS One']
['Int. Conf. Bioinformatics Biomedical Eng., iCBBE'
 'Int. Conf. Bioinformatics Biomed. Eng., iCBBE']
['BioEssays' 'Bioessays']
['Biotechnology' 'Biotechniques']


In [49]:
df.AbbrPubTitle = m

In [172]:
df.loc[df.Volume.isnull(),['CODEN','AbbrPubTitle','Volume','Issue','Month']].sample(20)

,CODEN,AbbrPubTitle,Volume,Issue,Month
1343,None,NaN,NaN,NaN,8
1740,None,NaN,NaN,NaN,NaN
1285,None,NaN,NaN,NaN,9
346,None,icm int con com med eng cme,NaN,NaN,NaN
1311,None,NaN,NaN,NaN,6
1681,None,NaN,NaN,NaN,NaN
1399,None,NaN,NaN,NaN,8
1747,None,NaN,NaN,NaN,NaN
135,None,chi con con ccc,NaN,NaN,9
1517,None,NaN,NaN,NaN,NaN


In [59]:
df.loc[df.Month.isnull() & df.AbbrPubTitle.notnull() & df.Volume.notnull()].shape

(665, 34)

In [60]:
df.shape

(2382, 34)

In [114]:
def g(row):
    if pd.isnull(row.Year):
        return np.nan
    if pd.isnull(row.Month):
        return np.nan
    return 12*(row.Year - 1970) + row.Month

In [76]:
print(len(df.CODEN.unique()))
print(len(df.AbbrPubTitle.unique()))

321
518


In [7]:
c = sqlalchemy.create_engine('sqlite:////home/matt/Datasets/Battelle Data/database/maincopy.db')
c2 = sqlalchemy.create_engine('sqlite:////home/matt/Datasets/Battelle Data/database/main.db')

In [44]:
imp.imputeCODEN(c,'main')

In [11]:
df = c2.execute("select CODEN from main")
df = df.fetchall()
df = pd.DataFrame(df)
print(df.shape)
df.isnull().sum()

(115099, 1)


0    73921
dtype: int64

In [12]:
df = c.execute("select CODEN from main")
df = df.fetchall()
df = pd.DataFrame(df)
print(df.shape)
df.isnull().sum()

(115099, 1)


0    50624
dtype: int64

In [8]:
imp.imputeTimeSlice(c,'main')

4 imputed
(4, 3)
18 imputed
(18, 3)
1 imputed
(1, 3)
3 imputed
(3, 3)
17 imputed
(17, 3)
6 imputed
(6, 3)
22 imputed
(22, 3)
32 imputed
(32, 3)
5 imputed
(5, 3)
1 imputed
(1, 3)
4 imputed
(4, 3)
1 imputed
(1, 3)
1 imputed
(1, 3)
2 imputed
(2, 3)
3 imputed
(3, 3)
37 imputed
(37, 3)
21 imputed
(21, 3)
2 imputed
(2, 3)
6 imputed
(6, 3)
2 imputed
(2, 3)
9 imputed
(9, 3)
14 imputed
(14, 3)
9 imputed
(9, 3)
8 imputed
(8, 3)
9 imputed
(9, 3)
1 imputed
(1, 3)
3 imputed
(3, 3)
1 imputed
(1, 3)
5 imputed
(5, 3)
1 imputed
(1, 3)
1 imputed
(1, 3)
5 imputed
(5, 3)
4 imputed
(4, 3)
2 imputed
(2, 3)
2 imputed
(2, 3)
24 imputed
(24, 3)
1 imputed
(1, 3)
1 imputed
(1, 3)
8 imputed
(8, 3)
16 imputed
(16, 3)
1 imputed
(1, 3)
3 imputed
(3, 3)
1 imputed
(1, 3)
1 imputed
(1, 3)
2 imputed
(2, 3)
2 imputed
(2, 3)
1 imputed
(1, 3)
3 imputed
(3, 3)
4 imputed
(4, 3)
36 imputed
(36, 3)
1 imputed
(1, 3)
185 imputed
(185, 3)
9 imputed
(9, 3)
1 imputed
(1, 3)
1 imputed
(1, 3)
8 imputed
(8, 3)
1 imputed
(1, 3)
1 imput

In [13]:
df = c2.execute("select TimeSlice from main")
df = df.fetchall()
df = pd.DataFrame(df)
print(df.shape)
df.isnull().sum()

(115099, 1)


0    55714
dtype: int64

In [14]:
df = c.execute("select TimeSlice from main")
df = df.fetchall()
df = pd.DataFrame(df)
print(df.shape)
df.isnull().sum()

(115099, 1)


0    55714
dtype: int64